In [5]:
!pip install langchain langchain-google-genai langchain-community chromadb pandas numpy python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 59.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 6.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ...

In [6]:
import numpy as np
import pandas as pd
import os
import json
import uuid
import logging
from kaggle_secrets import UserSecretsClient
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage, AIMessage
import zipfile
from IPython.display import FileLink

In [7]:
# Step 1: Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [8]:
# Step 2: Set up environment
try:
    user_secrets = UserSecretsClient()
    os.environ['GOOGLE_API_KEY'] = user_secrets.get_secret("GEMINI_API_KEY")
    logger.info("Gemini API key loaded from Kaggle Secrets")
except Exception as e:
    logger.error(f"Failed to load Gemini API key: {str(e)}")
    raise

SQUARE_ACCESS_TOKEN = 'EAAAl0ndxSs2S0Sowr7Yk9xaThBwM9jmLp0M9MTVocEBV-ELiMHbP0KLZPZuUpPz'

2025-05-05 06:32:40,758 - INFO - Gemini API key loaded from Kaggle Secrets


In [9]:
# Step 3: Create or load dataset
csv_path = '/kaggle/input/csv-api/square_payments_apis.csv'
alt_csv_path = '/kaggle/working/square_payments_apis.csv'

def create_dataset():
    data = {
        'API Name': [
            'Payments', 'Payments', 'Payments', 'Refunds', 'Customers',
        ],
        'Endpoint': [
            '/v2/payments',
            '/v2/payments/{payment_id}',
            '/v2/payments?begin_time={begin_time}&end_time={end_time}',
            '/v2/refunds',
            '/v2/customers',
        ],
        'Description': [
            'Create a payment',
            'Retrieve details for a specific payment',
            'List payments within a time range',
            'Create a refund for a payment',
            'Create a new customer',
        ],
        'Parameters': [
            'idempotency_key,source_id,accept_partial_authorization,amount_money',
            'payment_id',
            'begin_time,end_time',
            'idempotency_key,payment_id,amount_money',
            'idempotency_key,given_name,email_address',
        ],
        'Node Js': [
            """import { Client, Environment } from 'square';
async function main() {
    const client = new Client({
        environment: Environment.Sandbox,
        accessToken: 'ACCESS_TOKEN'
    });
    try {
        const response = await client.paymentsApi.createPayment({
            idempotencyKey: 'uuid',
            sourceId: 'cnon:gift-card-nonce-ok',
            acceptPartialAuthorization: true,
            amountMoney: { amount: BigInt(200), currency: 'USD' }
        });
        console.log(response.result);
    } catch (error) {
        console.error(error);
    }
}
main();""",
            """import { Client, Environment } from 'square';
async function main() {
    const client = new Client({
        environment: Environment.Sandbox,
        accessToken: 'ACCESS_TOKEN'
    });
    try {
        const response = await client.paymentsApi.getPayment('payment_id');
        console.log(response.result);
    } catch (error) {
        console.error(error);
    }
}
main();""",
            """import { Client, Environment } from 'square';
async function main() {
    const client = new Client({
        environment: Environment.Sandbox,
        accessToken: 'ACCESS_TOKEN'
    });
    try {
        const response = await client.paymentsApi.listPayments({
            beginTime: '2025-05-04T00:00:00Z',
            endTime: '2025-05-05T00:00:00Z'
        });
        console.log(response.result);
    } catch (error) {
        console.error(error);
    }
}
main();""",
            """import { Client, Environment } from 'square';
async function main() {
    const client = new Client({
        environment: Environment.Sandbox,
        accessToken: 'ACCESS_TOKEN'
    });
    try {
        const response = await client.refundsApi.createRefund({
            idempotencyKey: 'uuid',
            paymentId: 'payment_id',
            amountMoney: { amount: BigInt(100), currency: 'USD' }
        });
        console.log(response.result);
    } catch (error) {
        console.error(error);
    }
}
main();""",
            """import { Client, Environment } from 'square';
async function main() {
    const client = new Client({
        environment: Environment.Sandbox,
        accessToken: 'ACCESS_TOKEN'
    });
    try {
        const response = await client.customersApi.createCustomer({
            idempotencyKey: 'uuid',
            givenName: 'John Doe',
            emailAddress: 'john.doe@example.com'
        });
        console.log(response.result);
    } catch (error) {
        console.error(error);
    }
}
main();""",
        ],
        'Method': ['POST', 'GET', 'GET', 'POST', 'POST']
    }
    df = pd.DataFrame(data)
    output_csv = '/kaggle/working/square_payments_apis.csv'
    df.to_csv(output_csv, index=False)
    logger.info(f'Dataset created: {output_csv}')
    return output_csv

if os.path.exists(csv_path):
    logger.info(f'Using existing dataset: {csv_path}')
elif os.path.exists(alt_csv_path):
    csv_path = alt_csv_path
    logger.info(f'Using existing dataset: {csv_path}')
else:
    logger.warning(f'CSV not found at {csv_path} or {alt_csv_path}. Creating new dataset...')
    csv_path = create_dataset()

2025-05-05 06:32:40,779 - INFO - Using existing dataset: /kaggle/input/csv-api/square_payments_apis.csv


In [10]:
# Step 4: Set up Chroma VectorDB
try:
    df = pd.read_csv(csv_path)
    documents = []
    metadatas = []
    for _, row in df.iterrows():
        doc = f"{row['API Name']} - {row['Description']} - Endpoint: {row['Endpoint']}"
        documents.append(doc)
        metadatas.append({
            'api_name': row['API Name'],
            'endpoint': row['Endpoint'],
            'description': row['Description'],
            'parameters': row['Parameters'],
            'node_js': row['Node Js'],
            'method': row['Method']
        })

    embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    vector_store = Chroma.from_texts(
        texts=documents,
        embedding=embeddings,
        metadatas=metadatas,
        collection_name='square_apis',
        persist_directory='/kaggle/working/chroma_db'
    )
    logger.info('VectorDB setup complete in /kaggle/working/chroma_db')
except Exception as e:
    logger.error(f"Failed to set up VectorDB: {str(e)}")
    raise

2025-05-05 06:32:41,132 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-05-05 06:32:41,905 - INFO - VectorDB setup complete in /kaggle/working/chroma_db


In [23]:
# Step 5: Define tools
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from google.api_core.exceptions import ResourceExhausted

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=4, max=60),
    retry=retry_if_exception_type(ResourceExhausted),
    before_sleep=lambda retry_state: logger.info("Retrying retrieve_api_details due to ResourceExhausted error...")
)
def retrieve_api_details(query):
    try:
        results = vector_store.similarity_search(query, k=1)
        if results:
            logger.info(f"Retrieved API details for query: {query}")
            return json.dumps(results[0].metadata)
        logger.warning(f"No relevant APIs found for query: {query}")
        return json.dumps({'error': 'No relevant APIs found.'})
    except Exception as e:
        logger.error(f"Error retrieving API details: {str(e)}")
        return json.dumps({'error': f'Error retrieving API details: {str(e)}'})

def generate_code(api_details):
    try:
        api_details = json.loads(api_details)
        if 'error' in api_details:
            logger.error(f"API details contain error: {api_details['error']}")
            return json.dumps({'error': api_details['error']})

        api_name = api_details.get('api_name', 'Unknown')
        endpoint = api_details.get('endpoint', '')
        method = api_details.get('method', 'POST')
        parameters = api_details.get('parameters', '').split(',')
        node_js_code = api_details.get('node_js', '')

        # Determine API-specific inputs
        input_fields = []
        body_params = []
        query_params = []
        for param in parameters:
            param = param.strip()
            if method == 'POST':
                if param in ['source_id', 'amount_money', 'accept_partial_authorization', 'payment_id', 'given_name', 'email_address']:
                    input_fields.append(param)
                    body_params.append(param)
            elif method == 'GET':
                if param in ['payment_id', 'begin_time', 'end_time']:
                    input_fields.append(param)
                    query_params.append(param)

        # Frontend: React component
        frontend_inputs = ''
        frontend_state = ''
        frontend_body = ''
        default_values = {
            'source_id': 'cnon:gift-card-nonce-ok',
            'payment_id': 'J4cpcYKgzOfvQ7VO1ilH3CIpk7KZY',
            'begin_time': '2025-05-04T00:00:00Z',
            'end_time': '2025-05-05T00:00:00Z',
            'given_name': 'John Doe',
            'email_address': 'john.doe@example.com'
        }
        for param in input_fields:
            if param == 'amount_money':
                frontend_state += f'  const [amount, setAmount] = useState(200);\n'
                frontend_inputs += f"""
        <input
          type="number"
          value={{amount}}
          onChange={{(e) => setAmount(Number(e.target.value))}}
          placeholder="Amount (cents)"
        />
"""
                frontend_body += '      amountMoney: { amount: Number(amount), currency: "USD" },\n'
            elif param == 'accept_partial_authorization':
                frontend_state += f'  const [acceptPartial, setAcceptPartial] = useState(true);\n'
                frontend_inputs += f"""
        <label>
          <input
            type="checkbox"
            checked={{acceptPartial}}
            onChange={{(e) => setAcceptPartial(e.target.checked)}}
          />
          Accept Partial Authorization
        </label>
"""
                frontend_body += '      acceptPartialAuthorization: acceptPartial,\n'
            else:
                camel_param = ''.join(word.capitalize() for word in param.split('_'))
                default_value = default_values.get(param, '')
                frontend_state += f'  const [{param}, set{camel_param}] = useState("{default_value}");\n'
                frontend_inputs += f"""
        <input
          type="text"
          value={{{param}}}
          onChange={{(e) => set{camel_param}(e.target.value)}}
          placeholder="{param.replace('_', ' ').title()}"
        />
"""
                if param in body_params:
                    frontend_body += f'      {param}: {param},\n'

        fetch_body = ''
        fetch_url = f"'http://localhost:3001/{api_name.lower().replace(' ', '-')}'"
        if method == 'POST':
            fetch_body = f'body: JSON.stringify({{ {frontend_body.strip()} }}),'
        elif method == 'GET' and query_params:
            query_string = '&'.join(f'{param}={{{param}}}' for param in query_params)
            fetch_url = f"'http://localhost:3001/{api_name.lower().replace(' ', '-')}?{query_string}'"

        frontend_code = f"""
import React, {{ useState }} from 'react';

const {api_name.replace(' ', '')} = () => {{
{frontend_state}
  const [response, setResponse] = useState(null);
  const [error, setError] = useState(null);

  const handleAction = async () => {{
    try {{
      const res = await fetch({fetch_url}, {{
        method: '{method}',
        headers: {{ 'Content-Type': 'application/json' }},
        {fetch_body}
      }});
      if (!res.ok) throw new Error('Network response was not ok');
      const data = await res.json();
      setResponse(data);
    }} catch (err) {{
      setError(err.message);
    }}
  }};

  return (
    <div>
      <h2>{api_name}</h2>
      {frontend_inputs}
      <button onClick={{handleAction}}>Execute</button>
      {{response && <pre>{{JSON.stringify(response, null, 2)}}</pre>}}
      {{error && <p>Error: {{error}}</p>}}
    </div>
  );
}};

export default {api_name.replace(' ', '')};
"""

        # Backend: Express route
        backend_params = ''
        backend_call = ''
        if method == 'POST':
            backend_params = ', '.join(param for param in body_params if param != 'amount_money')
            if 'amount_money' in body_params:
                backend_params += (', ' if backend_params else '') + 'amountMoney'
            api_method = 'createPayment' if api_name == 'Payments' else 'createRefund' if api_name == 'Refunds' else 'createCustomer'
            backend_call = f"""
    const response = await client.{'paymentsApi' if api_name == 'Payments' else 'refundsApi' if api_name == 'Refunds' else 'customersApi'}.{api_method}({{
      {', '.join(param for param in body_params if param != 'amount_money')},
      {'amountMoney,' if 'amount_money' in body_params else ''}
      idempotencyKey: '{uuid.uuid4()}'
    }});
"""
        elif method == 'GET':
            if 'payment_id' in query_params:
                backend_params = 'paymentId'
                backend_call = f"""
    const response = await client.paymentsApi.getPayment(paymentId);
"""
            else:
                backend_params = ', '.join(query_params)
                backend_call = f"""
    const response = await client.paymentsApi.listPayments({{
      {', '.join(query_params)}
    }});
"""

        backend_code = f"""
const express = require('express');
const {{ Client, Environment }} = require('square');
const router = express.Router();

const client = new Client({{
  environment: Environment.Sandbox,
  accessToken: '{SQUARE_ACCESS_TOKEN}',
}});

router.{method.lower()}('/{api_name.lower().replace(' ', '-')}', async (req, res) => {{
  try {{
    const {{ {backend_params} }} = {'req.body' if method == 'POST' else 'req.query'};
    if (!{backend_params}) throw new Error('Missing required parameters');
    {backend_call}
    res.json(response.result);
  }} catch (error) {{
    res.status(500).json({{ error: error.message }});
  }}
}});

module.exports = router;
"""

        logger.info(f"Generated code for API: {api_name}")
        return json.dumps({
            'frontend': frontend_code,
            'backend': backend_code
        })
    except Exception as e:
        logger.error(f"Error generating code: {str(e)}")
        return json.dumps({'error': f'Error generating code: {str(e)}'})

In [33]:
# Step 6: Set up agent with ReAct framework
from langchain.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage, AIMessage

def setup_agent():
    try:
        # Define tools
        def retrieve_api_details(query):
            try:
                results = vector_store.similarity_search(query, k=1)
                if results:
                    logger.info(f"Retrieved API details for query: {query}")
                    return json.dumps(results[0].metadata)
                logger.warning(f"No relevant APIs found for query: {query}")
                return json.dumps({'error': 'No relevant APIs found.'})
            except Exception as e:
                logger.error(f"Error retrieving API details: {str(e)}")
                return json.dumps({'error': f'Error retrieving API details: {str(e)}'})

        def generate_code(api_details):
            try:
                api_details = json.loads(api_details)
                if 'error' in api_details:
                    logger.error(f"API details contain error: {api_details['error']}")
                    return json.dumps({'error': api_details['error']})

                api_name = api_details.get('api_name', 'Unknown')
                endpoint = api_details.get('endpoint', '')
                method = api_details.get('method', 'POST')
                parameters = api_details.get('parameters', '').split(',')

                # Simplified code generation for console output (mimics earlier logs)
                body_params = []
                query_params = []
                for param in parameters:
                    param = param.strip()
                    if method == 'POST':
                        if param in ['source_id', 'amount_money', 'accept_partial_authorization', 'payment_id']:
                            body_params.append(param)
                    elif method == 'GET':
                        if param in ['payment_id']:
                            query_params.append(param)

                frontend_body = ''
                default_values = {
                    'source_id': 'cnon:card-nonce-ok',
                    'payment_id': 'J4cpcYKgzOfvQ7VO1ilH3CIpk7KZY',
                    'amount_money': '{ amount: 100, currency: "USD" }',
                    'accept_partial_authorization': 'true'
                }
                for param in body_params:
                    frontend_body += f'      {param}: {default_values.get(param, f"REPLACE_WITH_{param.upper()}")},\n'
                for param in query_params:
                    frontend_body += f'      {param}: "{default_values.get(param, f"REPLACE_WITH_{param.upper()}")}",\n'

                fetch_url = f"'http://localhost:3001/{api_name.lower().replace(' ', '-')}'"
                if method == 'GET' and query_params:
                    query_string = '&'.join(f'{param}={{data.{param}}}' for param in query_params)
                    fetch_url = f"'http://localhost:3001/{api_name.lower().replace(' ', '-')}?{query_string}'"

                frontend_code = f"""
// Frontend code (JavaScript)
const data = {{
{frontend_body.strip()}
}};
fetch({fetch_url}, {{
    method: '{method}',
    headers: {{
        'Content-Type': 'application/json'
    }},
    {'body: JSON.stringify(data),' if method == 'POST' else ''}
}})
.then(response => response.json())
.then(data => console.log(data))
.catch(error => console.error('Error:', error));
"""

                backend_params = ''
                backend_call = ''
                if method == 'POST':
                    backend_params = ', '.join(param for param in body_params if param != 'amount_money')
                    if 'amount_money' in body_params:
                        backend_params += (', ' if backend_params else '') + 'amountMoney'
                    api_method = 'createPayment' if api_name == 'Payments' else 'createRefund'
                    backend_call = f"""
        const response = await client.{'paymentsApi' if api_name == 'Payments' else 'refundsApi'}.{api_method}({{
          {', '.join(param for param in body_params if param != 'amount_money')},
          {'amountMoney,' if 'amount_money' in body_params else ''}
          idempotencyKey: '{uuid.uuid4()}'
        }});
"""
                elif method == 'GET':
                    backend_params = 'paymentId'
                    backend_call = f"""
        const response = await client.paymentsApi.getPayment(paymentId);
"""

                backend_code = f"""
const express = require('express');
const {{ Client, Environment }} = require('square');
const router = express.Router();

const client = new Client({{
  environment: Environment.Sandbox,
  accessToken: 'YOUR_ACCESS_TOKEN'
}});

router.{method.lower()}('/{api_name.lower().replace(' ', '-')}', async (req, res) => {{
  try {{
    const {{ {backend_params} }} = {'req.body' if method == 'POST' else 'req.query'};
    if (!{backend_params}) throw new Error('Missing required parameters');
    {backend_call}
    res.json(response.result);
  }} catch (error) {{
    res.status(500).json({{ error: error.message }});
  }}
}});

module.exports = router;
"""

                return json.dumps({
                    'frontend': frontend_code,
                    'backend': backend_code
                })
            except Exception as e:
                logger.error(f"Error generating code: {str(e)}")
                return json.dumps({'error': f'Error generating code: {str(e)}'})

        tools = [
            Tool(
                name="retrieve_api_details",
                func=retrieve_api_details,
                description="Retrieves Square API details based on the user query."
            ),
            Tool(
                name="generate_code",
                func=generate_code,
                description="Generates frontend and backend code for a Square API based on API details."
            )
        ]

        # Set up memory (updated to avoid deprecation)
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            input_key="input",
            output_key="output"
        )

        # Define prompt
        prompt = PromptTemplate(
            input_variables=["input", "tools", "tool_names", "agent_scratchpad", "chat_history"],
            template="""You are an assistant that helps users integrate Square APIs into a payment system. Use the provided tools to retrieve API details and generate full-stack code. Follow these steps:

1. Understand the user's query about Square APIs.
2. Use the retrieve_api_details tool to get relevant API information.
3. Use the generate_code tool to create frontend and backend code based on the API details.
4. Return the generated code in JSON format with 'frontend' and 'backend' keys.
5. If an error occurs, return a JSON object with an 'error' key explaining the issue.

Always respond in this format:
Thought: [Your reasoning]
Action: [Tool to call or 'Finish']
Observation: [Result or output]

User query: {input}

Chat history: {chat_history}

Available tools: {tools}

Tool names: {tool_names}

Agent scratchpad: {agent_scratchpad}

Provide the response and, if code is generated, ensure it is returned as JSON."""
        )

        # Create agent
        agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            memory=memory,
            verbose=True,
            handle_parsing_errors=True
        )
        logger.info("Agent setup complete with Gemini LLM and ReAct framework")
        return agent_executor
    except Exception as e:
        logger.error(f"Failed to set up agent: {str(e)}")
        raise

agent_executor = setup_agent()

2025-05-05 07:28:50,162 - INFO - Agent setup complete with Gemini LLM and ReAct framework


In [35]:
# Step 7: Process user queries
import time
import json
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from google.api_core.exceptions import ResourceExhausted

# Define safe file name generator
def build_safe_filename(query):
    return "/kaggle/working/full_stack_code_" + query.lower().replace(" ", "_").replace("?", "").replace("/", "_") + ".html"

# Define test queries
queries = [
    'How do I create a payment using Square?',
    'How can I retrieve details of a specific payment?',
    'How do I create a refund for a payment?'
]

# Mock fallback for API details
def mock_retrieve_api_details(query):
    logger.info(f"Using mock retrieve_api_details for query: {query}")
    if "create a payment" in query.lower():
        return json.dumps({
            'api_name': 'Payments',
            'endpoint': '/v2/payments',
            'description': 'Create a payment',
            'parameters': 'source_id,amount_money,accept_partial_authorization',
            'node_js': 'Mock Node.js code',
            'method': 'POST'
        })
    elif "retrieve details" in query.lower():
        return json.dumps({
            'api_name': 'Payments',
            'endpoint': '/v2/payments/{payment_id}',
            'description': 'Retrieve details for a specific payment',
            'parameters': 'payment_id',
            'node_js': 'Mock Node.js code',
            'method': 'GET'
        })
    elif "create a refund" in query.lower():
        return json.dumps({
            'api_name': 'Refunds',
            'endpoint': '/v2/refunds',
            'description': 'Create a refund for a payment',
            'parameters': 'payment_id,amount_money',
            'node_js': 'Mock Node.js code',
            'method': 'POST'
        })
    return json.dumps({'error': 'No relevant APIs found.'})

# Mock fallback for code generation
def mock_generate_code(api_details):
    logger.info("Using mock generate_code")
    api_details = json.loads(api_details)
    if 'error' in api_details:
        return json.dumps({'error': api_details['error']})

    api_name = api_details.get('api_name', 'Unknown')
    method = api_details.get('method', 'POST')
    parameters = api_details.get('parameters', '').split(',')

    body_params = []
    query_params = []
    for param in parameters:
        param = param.strip()
        if method == 'POST':
            if param in ['source_id', 'amount_money', 'accept_partial_authorization', 'payment_id']:
                body_params.append(param)
        elif method == 'GET':
            if param in ['payment_id']:
                query_params.append(param)

    frontend_body = ''
    default_values = {
        'source_id': 'cnon:card-nonce-ok',
        'payment_id': 'J4cpcYKgzOfvQ7VO1ilH3CIpk7KZY',
        'amount_money': '{ amount: 100, currency: "USD" }',
        'accept_partial_authorization': 'true'
    }
    for param in body_params:
        value = default_values.get(param, f"REPLACE_WITH_{param.upper()}")
        frontend_body += f'      {param}: {value},\n'
    for param in query_params:
        value = default_values.get(param, f"REPLACE_WITH_{param.upper()}")
        frontend_body += f'      {param}: "{value}",\n'

    fetch_url = f"'http://localhost:3001/{api_name.lower().replace(' ', '-')}'"
    if method == 'GET' and query_params:
        query_string = '&'.join(f'{param}={{data.{param}}}' for param in query_params)
        fetch_url = f"'http://localhost:3001/{api_name.lower().replace(' ', '-')}?{query_string}'"

    frontend_code = f"""
// Frontend code (JavaScript)
const data = {{
{frontend_body.strip()}
}};
fetch({fetch_url}, {{
    method: '{method}',
    headers: {{
        'Content-Type': 'application/json'
    }},
    {'body: JSON.stringify(data),' if method == 'POST' else ''}
}})
.then(response => response.json())
.then(data => console.log(data))
.catch(error => console.error('Error:', error));
"""

    backend_params = ', '.join(param for param in body_params if param != 'amount_money')
    if 'amount_money' in body_params:
        backend_params += (', ' if backend_params else '') + 'amountMoney'
    api_method = 'createPayment' if api_name == 'Payments' else 'createRefund'

    backend_call = ""
    if method == 'POST':
        backend_call = f"""
    const response = await client.{'paymentsApi' if api_name == 'Payments' else 'refundsApi'}.{api_method}({{
      {', '.join(param for param in body_params if param != 'amount_money')},
      {'amountMoney,' if 'amount_money' in body_params else ''}
      idempotencyKey: 'UUID_PLACEHOLDER'
    }});
"""
    elif method == 'GET':
        backend_params = 'paymentId'
        backend_call = f"""
    const response = await client.paymentsApi.getPayment(paymentId);
"""

    backend_code = f"""
const express = require('express');
const {{ Client, Environment }} = require('square');
const router = express.Router();

const client = new Client({{
  environment: Environment.Sandbox,
  accessToken: 'YOUR_ACCESS_TOKEN'
}});

router.{method.lower()}('/{api_name.lower().replace(' ', '-')}', async (req, res) => {{
  try {{
    const {{ {backend_params} }} = {'req.body' if method == 'POST' else 'req.query'};
    if (!{backend_params}) throw new Error('Missing required parameters');
    {backend_call}
    res.json(response.result);
  }} catch (error) {{
    res.status(500).json({{ error: error.message }});
  }}
}});

module.exports = router;
"""

    return json.dumps({
        'frontend': frontend_code,
        'backend': backend_code
    })

# Main execution
output_files = []
for i, query in enumerate(queries):
    logger.info(f'Processing query: {query}')
    try:
        @retry(
            stop=stop_after_attempt(5),
            wait=wait_exponential(multiplier=1, min=10, max=120),
            retry=retry_if_exception_type(ResourceExhausted),
            before_sleep=lambda retry_state: logger.info(f"Retrying query '{query}' due to ResourceExhausted error...")
        )
        def execute_query(query):
            return agent_executor.invoke({'input': query})

        response = execute_query(query)
        output = response.get('output', '')

        code_dict = {}
        try:
            code_dict = json.loads(output)
        except:
            for message in memory.chat_memory.messages:
                if isinstance(message, AIMessage) and 'frontend' in message.content and 'backend' in message.content:
                    try:
                        code_dict = json.loads(message.content)
                        break
                    except:
                        continue

        if not code_dict or 'frontend' not in code_dict or 'backend' not in code_dict:
            logger.warning(f"No code generated for query: {query}. Falling back to mock implementation.")
            api_details = mock_retrieve_api_details(query)
            code_dict = json.loads(mock_generate_code(api_details))

        frontend_code = code_dict['frontend']
        backend_code = code_dict['backend']
        html_filename = build_safe_filename(query)

        print(f"Frontend:\n\n```javascript\n{frontend_code}\n```\n")
        print(f"Backend:\n\n```javascript\n{backend_code}\n```\n")
        print(f"(Code also saved to {html_filename})")

        output_html = f"""
<!DOCTYPE html>
<html>
<head>
  <title>Payment System - {query}</title>
</head>
<body>
  <h1>Full-Stack Code for {query}</h1>
  <h2>Frontend (JavaScript)</h2>
  <pre>{frontend_code}</pre>
  <h2>Backend (Node.js/Express)</h2>
  <pre>{backend_code}</pre>
</body>
</html>
"""
        with open(html_filename, 'w') as f:
            f.write(output_html)
        logger.info(f'Full-stack code saved to {html_filename}')
        output_files.append(html_filename)

        if i < len(queries) - 1:
            logger.info("Waiting 60 seconds before processing next query...")
            time.sleep(60)

    except Exception as e:
        logger.error(f"Failed to process query '{query}': {str(e)}")
        logger.info(f"Using mock implementation as fallback for query: {query}")
        api_details = mock_retrieve_api_details(query)
        code_dict = json.loads(mock_generate_code(api_details))

        frontend_code = code_dict['frontend']
        backend_code = code_dict['backend']
        html_filename = build_safe_filename(query)

        print(f"Frontend:\n\n```javascript\n{frontend_code}\n```\n")
        print(f"Backend:\n\n```javascript\n{backend_code}\n```\n")
        print(f"(Code also saved to {html_filename})")

        output_html = f"""
<!DOCTYPE html>
<html>
<head>
  <title>Payment System - {query}</title>
</head>
<body>
  <h1>Full-Stack Code for {query}</h1>
  <h2>Frontend (JavaScript)</h2>
  <pre>{frontend_code}</pre>
  <h2>Backend (Node.js/Express)</h2>
  <pre>{backend_code}</pre>
</body>
</html>
"""
        with open(html_filename, 'w') as f:
            f.write(output_html)
        logger.info(f'Full-stack code (mock) saved to {html_filename}')
        output_files.append(html_filename)

        if i < len(queries) - 1:
            logger.info("Waiting 60 seconds before processing next query...")
            time.sleep(60)


2025-05-05 07:31:02,834 - INFO - Processing query: How do I create a payment using Square?




> Entering new AgentExecutor chain...
Thought: The user wants to create a payment using Square. I need to retrieve the relevant API details and generate code for it.

Action: retrieve_api_details(query="Create a payment using Square")
Invalid Format: Missing 'Action Input:' after 'Action:'Thought: The user wants to create a payment using Square. I need to retrieve the relevant API details and generate code for it.

Action: retrieve_api_details(query="Create a payment using Square")
Invalid Format: Missing 'Action Input:' after 'Action:'Thought: The user wants to create a payment using Square. I need to retrieve the relevant API details and generate code for it.

Action: retrieve_api_details(query="Create a payment using Square")
Invalid Format: Missing 'Action Input:' after 'Action:'

2025-05-05 07:31:05,850 - WARNING - Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 54
}
].
2025-05-05 07:31:07,900 - INFO - Retrying query 'How do I create a payment using Square?' due to ResourceExhausted error...




> Entering new AgentExecutor chain...
Thought: The user wants to create a payment using Square. I need to retrieve the relevant API details and generate code for it.

Action: retrieve_api_details(query="Create a payment using Square")
Invalid Format: Missing 'Action Input:' after 'Action:'Thought: The user wants to create a payment using Square. I need to retrieve the relevant API details and generate code for it.

Action: retrieve_api_details(query="Create a payment using Square")
Invalid Format: Missing 'Action Input:' after 'Action:'Thought: The user wants to create a payment using Square. I need to retrieve the relevant API details and generate code for it.

Action: retrieve_api_details(query="Create a payment using Square")
Invalid Format: Missing 'Action Input:' after 'Action:'

2025-05-05 07:31:21,189 - WARNING - Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 38
}
].
2025-05-05 07:31:23,226 - INFO - Retrying query 'How do I create a payment using Square?' due to ResourceExhausted error...


...

2025-05-05 07:35:37,603 - WARNING - Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 22
}
].
2025-05-05 07:35:39,651 - INFO - Retrying query 'How do I create a refund for a payment?' due to ResourceExhausted error...




> Entering new AgentExecutor chain...


2025-05-05 07:35:49,704 - WARNING - Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 10
}
].
2025-05-05 07:35:51,752 - ERROR - Failed to process query 'How do I create a refund for a payment?': RetryError[<Future at 0x796b5c13dc90 state=finished raised ResourceExhausted>]
2025-05-05 07:35:51,754 - INFO - Using mock implementation as fallback for query: How do I create a refund for a payment?
2025-05-05 07:35:51,755 - INFO - Using mock retrieve_api_details for query: How do I create a refund for a payment?
2025-05-05 07:35:51,755 - INFO - Using mock generate_code


Frontend:

```javascript

// Frontend code (JavaScript)
const data = {
payment_id: J4cpcYKgzOfvQ7VO1ilH3CIpk7KZY,
      amount_money: { amount: 100, currency: "USD" },
};
fetch('http://localhost:3001/refunds', {
    method: 'POST',
    headers: {
        'Content-Type': 'application/json'
    },
    body: JSON.stringify(data),
})
.then(response => response.json())
.then(data => console.log(data))
.catch(error => console.error('Error:', error));

```

Backend:

```javascript

const express = require('express');
const { Client, Environment } = require('square');
const router = express.Router();

const client = new Client({
  environment: Environment.Sandbox,
  accessToken: 'YOUR_ACCESS_TOKEN'
});

router.post('/refunds', async (req, res) => {
  try {
    const { payment_id, amountMoney } = req.body;
    if (!payment_id, amountMoney) throw new Error('Missing required parameters');
    
    const response = await client.refundsApi.createRefund({
      payment_id,
      amountMoney,
      i

2025-05-05 07:35:51,758 - INFO - Full-stack code (mock) saved to /kaggle/working/full_stack_code_how_do_i_create_a_refund_for_a_payment.html


In [14]:
# Step 8: Create a focused zip file
zip_name = 'square_payment_system.zip'
try:
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Include dataset
        zipf.write(csv_path, os.path.basename(csv_path))
        # Include generated HTML files
        for html_file in output_files:
            zipf.write(html_file, os.path.basename(html_file))
    logger.info(f'Output files zipped to {zip_name}')
    FileLink(zip_name)
except Exception as e:
    logger.error(f"Failed to create zip file: {str(e)}")
    raise

/kaggle/working/A_12.zip